<a href="https://colab.research.google.com/github/musicjae/Korean-NLP/blob/main/utils/TextRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reference  
[1] https://excelsior-cjh.tistory.com/93

- 텍스트랭크: 페이지랭크를 기반으로 만든 알고리즘. 문서 내에서 문장 또는 단어 간 중요도를 고려하여 랭킹을 계산
  - 페이지 랭크: 중요도가 높은 웹 페이지는 다른 많은 페이지로부터 링크를 받는다는 점에서 착안.
    - PR(A) = (1-d) + d (PR(T1)/C(T1) + … + PR(Tn)/C(Tn))  
        - PR(T1): T1의 PageRank value
        - C(T1): T1이 갖는 링크의 개수  
        - **함의**:‘어떤 페이지 A의 페이지 랭크는 그 페이지를 인용하고 있는 다른 페이지 T1, T2, T3, .. 가 가진 페이지 랭크를 정규화시킨 값의 합'
        - 보통 d = 0.85 
            - 85% 확률로 다른 페이지 클릭 / 15% 확률로 순간 클릭을 멈추고 그 페이지 살펴봄
        - **문제점**: 위 방식 채택 시, PR의 합산은 모든 페이지 개수인 N이 됨.
        - **위키피디아 제안**: 위 문제점 해소 위해, PR(A) = (1-d)/N + d (PR(T1)/C(T1) + … + PR(Tn)/C(Tn))가 적절한 식.

- TR: $TR(Vi)=(1−d)+d∗∑_{Vj∈In(Vi)}\frac{w_{ji}}{∑_{Vk∈Out(Vj)}w_{jk}}TR(Vj)$

- TR(Vi): 문장 또는 단어(V)i에 대한 TextRank값
- $w_{ij}$: 문장 또는 단어 i 와 j 사이의 가중치
- d:damping factor, PageRank에서 웹 서핑을 하는 사람이 해당 페이지를 만족하지 못하고 다른페이지로 이동하는 확률, TextRank에서도 그 값을 그대로 사용(0.85로 설정)
- TextRank TR(Vi)를 계산 한 뒤 높은 순으로 정렬



In [1]:
!pip install konlpy
!pip install newspaper3k

from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np


In [2]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.twitter = Okt()
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자","아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]

    def url2sentences(self, url):
        article = Article(url, language='ko')
        article.download()
        article.parse()
        sentences = self.kkma.sentences(article.text)

        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''

        return sentences

    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences


    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence)) if noun not in self.stopwords and len(noun) > 1]))
        return nouns



## TF-IDF

- TF: 특정 단어가 문서 내에 얼만큼 나타나는지 수치
- IDF: 전체 문서 수 / 해당 단어가 나타난 문서 수
  - DF의 역수 사용 이유: DF가 가장 큰 값을 1로 만들어주기 위함 (확률 처리)

- Adjacency Mat: Correlation Mat는 sen-term Mat의 전치와 원행렬을 곱하여 구함. 이것이 곧 Adj mat

In [3]:
class GraphMatrix(object):
    
    
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []


    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence


    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}


In [4]:
class Rank(object):
    
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1

        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}


In [5]:
class TextRank(object):


    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)


    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    
    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords


In [ ]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=055&aid=0000877970'
textrank = TextRank(url)
for row in textrank.summarize(3):
    print(row)
    print()
print('keywords :',textrank.keywords())